# NB-HI8: Plot of PET and dryness over three 30yr period for each GCM

## module loading...

In [ ]:
%matplotlib inline  
import os
import sys
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from timeit import default_timer as timer

print("\nThe Python version: %s.%s.%s" % sys.version_info[:3])
print(xr.__name__, xr.__version__)
print(pd.__name__, pd.__version__)

mpl.rcParams['xtick.labelsize']='small'

In [ ]:
from cmip5_oconus import print_date
print_date()

from cmip5_oconus.data_catalog import load_monthly_historical_hydro_datasets, resample_data
from cmip5_oconus.data_catalog import load_monthly_cmip5_hydro_datasets
from cmip5_oconus.plot import custom_div_cmap, MidpointNormalize

In [ ]:
#from dask_jobqueue import PBSCluster
#cluster = PBSCluster(processes=9, threads=4, memory="108GB",
#                  walltime='00:40:00')
#cluster.scale(jobs=3)


In [ ]:
#from dask.distributed import Client
#client = Client(cluster)

In [ ]:
#client

-------------------------
## Setup 

In [ ]:
figSave = True

In [ ]:
region           = 'HI'
gcms             = ["ACCESS1-3","CanESM2","CCSM4","CSIRO-Mk3-6-0","GFDL-ESM2M","HadGEM2-ES","inmcm4","MIROC5","MPI-ESM-MR","MRI-CGCM3"]
period_hist      = slice('1970-01-01', '1999-12-31')
period_10yr_hist = slice('1990-01-01', '1999-12-31')
pet_vars  = ['PRCP', 'OUT_PET_NATVEG']

## Load the data

In [ ]:
xr.set_options(file_cache_maxsize=12000)
histo_data = load_monthly_historical_hydro_datasets(models=gcms, variables=pet_vars, region=region, parallel=False)

## 30 year (1970-1999) annual mean

In [ ]:
%%time
histo_means = {}
for name in ['obs', 'gcm']:
    histo_means[name] = resample_data(histo_data[name].sel(time=period_hist), region=region, freq='AS').mean('time').compute()

## 10 year (1990-1999) annual mean

In [ ]:
histo_10yr_means = {}
for name in ['obs', 'gcm']:
    histo_10yr_means[name] = resample_data(histo_data[name].sel(time=period_10yr_hist), region=region, freq='AS').mean('time').compute()

## Additional process - compute dryness index, and cleanup

In [ ]:
# compute dryness
for name in ['obs', 'gcm']:
    histo_means[name]['dryness'] = histo_means[name]['OUT_PET_NATVEG']/histo_means[name]['PRCP']
    histo_10yr_means[name]['dryness'] = histo_10yr_means[name]['OUT_PET_NATVEG']/histo_10yr_means[name]['PRCP']

### Plot

In [ ]:
# create colormap
# ---------------

# create a colormap that consists of
# - 1/5 : custom colormap, ranging from white to the first color of the colormap
# - 4/5 : existing colormap

# set upper part: 4 * 256/4 entries
upper = mpl.cm.Spectral_r(np.arange(256))

# set lower part: 1 * 256/4 entries
# - initialize all entries to 1 to make sure that the alpha channel (4th column) is 1
lower = np.ones((int(256/4),4))
# - modify the first three columns (RGB):
#   range linearly between white (1,1,1) and the first color of the upper colormap
for i in range(3):
    lower[:,i] = np.linspace(0.9, upper[0,i], lower.shape[0])

# combine parts of colormap
cmap = np.vstack(( lower, upper ))

# convert to matplotlib colormap
cmap = mpl.colors.ListedColormap(cmap, name='myColorMap', N=cmap.shape[0])

## 30 year (1970-1999) annual mean plot

In [ ]:
var="OUT_PET_NATVEG"

cm = {'mean': cmap,
      'std' : custom_div_cmap(mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

sfmt = mpl.ticker.ScalarFormatter(useMathText=True)
sfmt.set_powerlimits((0, 0))

cbar_kwrgs = {'mean':   {"orientation":"horizontal", "shrink":0.990, "pad":0.03, 'format':sfmt},
              'std':    {"orientation":"horizontal", "shrink":0.990, "pad":0.03}}

style_kwargs = {'mean': {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['mean']},
                'std':  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['std']}}

kwargs = {'mean': {'levels': 21, 'vmin':0, 'vmax':2000, 'cmap':cm['mean'], 'extend':'max', **style_kwargs['mean']},
          'std':  {'levels': 6, 'vmin':0, 'vmax':50,  'cmap':cm['std'],  'extend':'max', **style_kwargs['std']}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5)) # used to be figsize=(14,11)
plt.subplots_adjust(left=0.012, bottom=0.0125, right=0.99, top=0.935, hspace=0.125, wspace=0.075)

for ix, gcm in enumerate(gcms+['UH (1990-1999)','GCM ensemble std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        histo_means['gcm'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        histo_means['obs'][var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        histo_means['gcm'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=14)
    
#plt.tight_layout()
fig.suptitle('1970-1999 mean annual PET [mm/yr]', fontsize=14, y=0.985)
if figSave:
    plt.savefig(f'NB-HI8_Fig1_annual_mean_{var}_hist.png', dpi=300)

In [ ]:
var="dryness"

cm = {'mean': cmap,
      'std' : custom_div_cmap(mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

sfmt = mpl.ticker.ScalarFormatter(useMathText=True)
sfmt.set_powerlimits((0, 0))

cbar_kwrgs = {'mean':   {"orientation":"horizontal", "shrink":0.990, "pad":0.03, 'format':sfmt},
              'std':    {"orientation":"horizontal", "shrink":0.990, "pad":0.03}}

style_kwargs = {'mean': {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['mean']},
                'std':  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['std']}}

kwargs = {'mean': {'levels': 11, 'vmin':0, 'vmax':2.5, 'cmap':cm['mean'], 'extend':'max', **style_kwargs['mean']},
          'std':  {'levels': 6, 'vmin':0, 'vmax':0.5, 'cmap':cm['std'],  'extend':'max', **style_kwargs['std']}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5)) # used to be figsize=(14,11)
plt.subplots_adjust(left=0.012, bottom=0.0125, right=0.99, top=0.935, hspace=0.125, wspace=0.075)

for ix, gcm in enumerate(gcms+['UH (1990-1999)','GCM ensemble std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        histo_means['gcm'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        histo_means['obs'][var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        histo_means['gcm'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=14)

fig.suptitle('1970-1999 mean annual dryness index [-]', fontsize=14, y=0.985)
if figSave:
    plt.savefig(f'NB-HI8_Fig2_annual_mean_{var}_hist.png', dpi=300)

## 10 year (1990-1999) annual mean plot

In [ ]:
var="OUT_PET_NATVEG"

cm = {'mean': cmap,
      'std' : custom_div_cmap(mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

sfmt = mpl.ticker.ScalarFormatter(useMathText=True)
sfmt.set_powerlimits((0, 0))

cbar_kwrgs = {'mean':   {"orientation":"horizontal", "shrink":0.990, "pad":0.03, 'format':sfmt},
              'std':    {"orientation":"horizontal", "shrink":0.990, "pad":0.03}}

style_kwargs = {'mean': {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['mean']},
                'std':  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['std']}}

kwargs = {'mean': {'levels': 21, 'vmin':0, 'vmax':2000, 'cmap':cm['mean'], 'extend':'max', **style_kwargs['mean']},
          'std':  {'levels': 6, 'vmin':0, 'vmax':50,  'cmap':cm['std'],  'extend':'max', **style_kwargs['std']}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5)) # used to be figsize=(14,11)
plt.subplots_adjust(left=0.012, bottom=0.0125, right=0.99, top=0.935, hspace=0.125, wspace=0.075)

for ix, gcm in enumerate(gcms+['UH (1990-1999)','GCM ensemble std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        histo_10yr_means['gcm'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        histo_10yr_means['obs'][var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        histo_10yr_means['gcm'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=14)
    
#plt.tight_layout()
fig.suptitle('1970-1999 mean annual PET [mm/yr]', fontsize=14, y=0.985)
if figSave:
    plt.savefig(f'NB-HI8_Fig3_annual_mean_{var}_10yr_hist.png', dpi=300)

In [ ]:
var="dryness"

cm = {'mean': cmap,
      'std' : custom_div_cmap(mincol='xkcd:light grey', midcol='xkcd:light light blue', maxcol='xkcd:blue')}

sfmt = mpl.ticker.ScalarFormatter(useMathText=True)
sfmt.set_powerlimits((0, 0))

cbar_kwrgs = {'mean':   {"orientation":"horizontal", "shrink":0.990, "pad":0.03, 'format':sfmt},
              'std':    {"orientation":"horizontal", "shrink":0.990, "pad":0.03}}

style_kwargs = {'mean': {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['mean']},
                'std':  {'add_labels': False, 'xticks':[], 'yticks':[], 'cbar_kwargs': cbar_kwrgs['std']}}

kwargs = {'mean': {'levels': 11, 'vmin':0, 'vmax':2.5, 'cmap':cm['mean'], 'extend':'max', **style_kwargs['mean']},
          'std':  {'levels': 6, 'vmin':0, 'vmax':0.5, 'cmap':cm['std'],  'extend':'max', **style_kwargs['std']}}

fig, axes = plt.subplots(nrows=3, ncols=4, sharex=True, sharey=True, figsize=(11.375, 10.5)) # used to be figsize=(14,11)
plt.subplots_adjust(left=0.012, bottom=0.0125, right=0.99, top=0.935, hspace=0.125, wspace=0.075)

for ix, gcm in enumerate(gcms+['UH','GCM ensemble std']):
    row = ix // 4
    col = ix % 4
    if ix < 10:
        histo_10yr_means['gcm'][var].sel(gcm=gcm).plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 10:
        histo_10yr_means['obs'][var].plot.pcolormesh(ax=axes[row, col], **kwargs['mean'])
    elif ix == 11:
        histo_10yr_means['gcm'].std(dim='gcm')[var].plot.pcolormesh(ax=axes[row, col], **kwargs['std'])
    axes[row, col].set_title(gcm, fontsize=14)
    
fig.suptitle('1990-1999 mean annual dryness [-]', fontsize=14, y=0.985)
if figSave:
    plt.savefig(f'NB-HI8_Fig4_annual_mean_{var}_10yr_hist.png', dpi=300)

### Below is for checking random things